In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\merged_data.csv")

# age from birth_year
df['age']=2025-df['year_of_birth']

# device type(e.g high-end vd low-end based on brand/model)
high_end_brands =['Apple','Samsung','Oneplus']
df['device_segment']= df['brand_name'].apply(lambda x: 'High-End' if x in high_end_brands else 'Others')

# average weekly Revenue
revenue_df = revenue_df = pd.read_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\Revenue_data.csv")
avg_revenue = revenue_df.groupby('msisdn')['revenue_usd'].mean().reset_index()
avg_revenue.columns=['msisdn','avg_weekly_revenue']
df = df.merge(avg_revenue, on='msisdn',how="left")

# flag missing gender
df['is_gender_missing'] = df['gender'].isna().astype(int)

# total weekly revenue or revenue category
df['revenue_category'] =pd.qcut(df['avg_weekly_revenue'],q=4,labels=['Low','Medium','High','Very High'])

df.to_csv(r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv",
        sep=None , engine ='python',nrows=5  )
print(df.columns)

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type int64

In [2]:
import pandas as pd
import numpy as np

# Define your input and output paths
merged_path = r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\merged_data.csv"
revenue_path = r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\Revenue_data.csv"
output_path = r"C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv"

# Load average revenue first (smaller dataset)
revenue_df = pd.read_csv(revenue_path)
avg_revenue = revenue_df.groupby('msisdn')['revenue_usd'].mean().reset_index()
avg_revenue.columns = ['msisdn', 'avg_weekly_revenue']

# Brands considered high-end
high_end_brands = ['Apple', 'Samsung', 'Oneplus']

# Process merged_data.csv in chunks
chunk_size = 100_000
processed_chunks = []

for chunk in pd.read_csv(merged_path, chunksize=chunk_size, low_memory=False):
    # age from birth_year
    chunk['age'] = 2025 - chunk['year_of_birth']

    # device type
    chunk['brand_name'] = chunk['brand_name'].str.strip().str.title()
    chunk['device_segment'] = chunk['brand_name'].apply(
        lambda x: 'High-End' if x in high_end_brands else 'Others'
    )

    # merge avg weekly revenue
    chunk = chunk.merge(avg_revenue, on='msisdn', how='left')

    # flag missing gender
    chunk['is_gender_missing'] = chunk['gender'].isna().astype(int)

     # Check if there are enough non-NaN values to perform qcut
    if chunk['avg_weekly_revenue'].notna().sum() >= 4:
        chunk['revenue_category'] = pd.qcut(
            chunk['avg_weekly_revenue'],
            q=4,
            labels=['Low', 'Medium', 'High', 'Very High'],
            duplicates='drop'  # optional: prevent bin edge duplication
        )
    else:
        chunk['revenue_category'] = np.nan


    processed_chunks.append(chunk)

# Combine all processed chunks into one DataFrame
final_df = pd.concat(processed_chunks)

# Save final output to CSV
final_df.to_csv(output_path, index=False)
print(" Data processing complete. File saved to:", output_path)


C:\Users\harsh\AppData\Local\Temp\ipykernel_11176\1761362228.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(processed_chunks)


 Data processing complete. File saved to: C:\Users\harsh\OneDrive\Desktop\data analysis\Revenue_leakage and KPI analytics project\data\cleaned\engineered_data.csv
